In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import collections
from matplotlib.colors import ListedColormap
import re


plt.rc('font', family='AppleGothic') 
plt.rcParams['axes.unicode_minus'] =False

In [2]:
product = pd.read_csv("Zigzag_products.csv")
review = pd.read_csv("Zigzag_reviews.csv")

In [3]:
review['리뷰 코멘트'] = review['리뷰 코멘트'].str.replace('\n', ' ', regex=False)
review.head(3)

,리뷰순서,카테고리,브랜드,상품명,작성자,작성 날짜,옵션,사이즈,퀄리티,색감,정보,리뷰 점수,리뷰 코멘트,배송형태,포토리뷰,상품ID
0,1,의류,퍼빗,퍼빗 [랩티1등🏆/누적5만장🩰/모델전색소장💗/평점4.8극찬💭/기모ver.] 단독제작...,솔링이,25.02.01,-3kg블랙,정사이즈예요,아주 만족해요,화면과 비슷해요,158cm42kg상의 44,5,[158/42kg) 이 옷이 유독 슬림해보이는 거 같아요 ㅎㅎ 여리여리한 느낌이 ...,직진,있음,130108350
1,2,의류,퍼빗,퍼빗 [랩티1등🏆/누적5만장🩰/모델전색소장💗/평점4.8극찬💭/기모ver.] 단독제작...,콩콩땅,25.01.30,발렛베이지,정사이즈예요,아주 만족해요,화면과 비슷해요,158cm50kg상의 44,5,"⭐️ 스펙 : 158cm/50kg , 상체 보다 하체 발달형 ⭐️ 핏, 사이즈 ...",직진,있음,130108350
2,3,의류,퍼빗,퍼빗 [랩티1등🏆/누적5만장🩰/모델전색소장💗/평점4.8극찬💭/기모ver.] 단독제작...,미소세계,25.01.26,발렛베이지,정사이즈예요,아주 만족해요,화면과 비슷해요,160cm53kg상의 55,5,발레코어룩의 정석인 아이템 발견했네여!!!ㅎㅎ 완전 여리여리해보이는 효과가 있어욤ㅠ...,일반,있음,130108350


In [4]:
coment = review['리뷰 코멘트']

In [5]:
#불용어 제거
import re
def clean_str(text):
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '<[^>]*>'         # HTML 태그 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '[^\w\s]'         # 특수기호제거
    text = re.sub(pattern=pattern, repl='', string=text)
    return text   

coments = []
for i in coment:
    if isinstance(i, str):  # i가 문자열일 경우에만 처리
        a = clean_str(i)
        coments.append(a)
    else:
        coments.append('')  # 문자열이 아닌 값은 빈 문자열로 대체

In [6]:
coments[0]

'15842kg  이 옷이 유독 슬림해보이는 거 같아요  여리여리한 느낌이 최고에요   엄지손 뚫려 있어서 여리여리한 느낌도 나고 브이넥이라 어깨 라인이 예쁘게 나와서 입었을 때 확실히 더 여성스럽고 예뻐요     오픈숄더로도 입을 더보기지그재그 체험단 이벤트로 상품을 무료로 제공받아 작성한 리뷰입니다'

In [7]:
len(coments)

113091

In [8]:
len(review['리뷰 코멘트'])

113091

In [9]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer  ## 방법 2개
# 키워드 길이 지정
n_gram_range = (1, 1)

### 1. count-vectorizer

In [12]:
# 리뷰하나씩 키워드로 분리(count-vectorizer)
a= len(coments) # 불용어 제거한 리뷰

token_review_list = []
n_gram_range = (1, 2)  # 예시로 1-gram, 2-gram 사용

for i in range(0, a):  # 리뷰 하나씩 처리
    if not coments[i].strip():  # 빈 문자열이면 건너뛰기
        token_review_list.append([])  # 빈 리스트 추가
        continue
    
    review_vectorized = CountVectorizer(ngram_range=n_gram_range).fit([coments[i]])
    token_review = review_vectorized.get_feature_names_out()
    token_review_list.append(token_review)

# 결과 확인
print(token_review_list[0])

['15842kg' '15842kg 옷이' '같아요' '같아요 여리여리한' '나고' '나고 브이넥이라' '나와서' '나와서 입었을'
 '느낌도' '느낌도 나고' '느낌이' '느낌이 최고에요' '더보기지그재그' '더보기지그재그 체험단' '뚫려' '뚫려 있어서'
 '라인이' '라인이 예쁘게' '리뷰입니다' '무료로' '무료로 제공받아' '브이넥이라' '브이넥이라 어깨' '상품을'
 '상품을 무료로' '슬림해보이는' '슬림해보이는 같아요' '어깨' '어깨 라인이' '엄지손' '엄지손 뚫려' '여리여리한'
 '여리여리한 느낌도' '여리여리한 느낌이' '여성스럽고' '여성스럽고 예뻐요' '예뻐요' '예뻐요 오픈숄더로도' '예쁘게'
 '예쁘게 나와서' '오픈숄더로도' '오픈숄더로도 입을' '옷이' '옷이 유독' '유독' '유독 슬림해보이는' '이벤트로'
 '이벤트로 상품을' '입었을' '입었을 확실히' '입을' '입을 더보기지그재그' '있어서' '있어서 여리여리한' '작성한'
 '작성한 리뷰입니다' '제공받아' '제공받아 작성한' '체험단' '체험단 이벤트로' '최고에요' '최고에요 엄지손' '확실히'
 '확실히 여성스럽고']


In [15]:
len(token_review_list)

113091

In [16]:
review['review_count'] = token_review_list

### 2. tfidf-vectorizer

In [17]:
# 리뷰하나씩 키워드로 분리(tfidf-vectorizer)
a= len(coments) # 불용어 제거한 리뷰


token_review_list = []
for i in range(0, a):  # 리뷰 하나씩 처리
    if not coments[i].strip():  # 빈 문자열이면 건너뛰기
        token_review_list.append([])  # 빈 리스트 추가
        continue
    
    review_vectorized = TfidfVectorizer(ngram_range=n_gram_range).fit([coments[i]])
    token_review = review_vectorized.get_feature_names_out()
    token_review_list.append(token_review)

# 결과 확인
print(token_review_list[0])

['15842kg' '15842kg 옷이' '같아요' '같아요 여리여리한' '나고' '나고 브이넥이라' '나와서' '나와서 입었을'
 '느낌도' '느낌도 나고' '느낌이' '느낌이 최고에요' '더보기지그재그' '더보기지그재그 체험단' '뚫려' '뚫려 있어서'
 '라인이' '라인이 예쁘게' '리뷰입니다' '무료로' '무료로 제공받아' '브이넥이라' '브이넥이라 어깨' '상품을'
 '상품을 무료로' '슬림해보이는' '슬림해보이는 같아요' '어깨' '어깨 라인이' '엄지손' '엄지손 뚫려' '여리여리한'
 '여리여리한 느낌도' '여리여리한 느낌이' '여성스럽고' '여성스럽고 예뻐요' '예뻐요' '예뻐요 오픈숄더로도' '예쁘게'
 '예쁘게 나와서' '오픈숄더로도' '오픈숄더로도 입을' '옷이' '옷이 유독' '유독' '유독 슬림해보이는' '이벤트로'
 '이벤트로 상품을' '입었을' '입었을 확실히' '입을' '입을 더보기지그재그' '있어서' '있어서 여리여리한' '작성한'
 '작성한 리뷰입니다' '제공받아' '제공받아 작성한' '체험단' '체험단 이벤트로' '최고에요' '최고에요 엄지손' '확실히'
 '확실히 여성스럽고']


In [18]:
len(token_review_list)

113091

In [19]:
review['review_tfidf'] = token_review_list

In [20]:
review.head(2)

,리뷰순서,카테고리,브랜드,상품명,작성자,작성 날짜,옵션,사이즈,퀄리티,색감,정보,리뷰 점수,리뷰 코멘트,배송형태,포토리뷰,상품ID,review_count,review_tfidf
0,1,의류,퍼빗,퍼빗 [랩티1등🏆/누적5만장🩰/모델전색소장💗/평점4.8극찬💭/기모ver.] 단독제작...,솔링이,25.02.01,-3kg블랙,정사이즈예요,아주 만족해요,화면과 비슷해요,158cm42kg상의 44,5,[158/42kg) 이 옷이 유독 슬림해보이는 거 같아요 ㅎㅎ 여리여리한 느낌이 ...,직진,있음,130108350,"[15842kg, 15842kg 옷이, 같아요, 같아요 여리여리한, 나고, 나고 브...","[15842kg, 15842kg 옷이, 같아요, 같아요 여리여리한, 나고, 나고 브..."
1,2,의류,퍼빗,퍼빗 [랩티1등🏆/누적5만장🩰/모델전색소장💗/평점4.8극찬💭/기모ver.] 단독제작...,콩콩땅,25.01.30,발렛베이지,정사이즈예요,아주 만족해요,화면과 비슷해요,158cm50kg상의 44,5,"⭐️ 스펙 : 158cm/50kg , 상체 보다 하체 발달형 ⭐️ 핏, 사이즈 ...",직진,있음,130108350,"[158cm50kg, 158cm50kg 상체, 감싸줘요, 감싸줘요 재질이, 그자체,...","[158cm50kg, 158cm50kg 상체, 감싸줘요, 감싸줘요 재질이, 그자체,..."


In [225]:
review['clean_count'] = review['review_count'].apply(lambda x: ' '.join(x))
review['clean_tfidf'] = review['review_tfidf'].apply(lambda x: ' '.join(x))

In [228]:
review.to_csv("review_전처리.csv", index=False, encoding='utf-8-sig')